In [ ]:
# ============================================================
# LP-IV Assignment: Continuous Bag of Words (CBOW)
# Works for any input text (Document 1 / 2 / 3)
# ============================================================

# a. Data Preparation
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

# ------------------------------------------------------------
# 📄 STEP 1: Add or Load Your Document Text
# ------------------------------------------------------------

# Option 1: Paste text directly here
data = """
PASTE YOUR DOCUMENT TEXT HERE
"""

# --- OR ---
# Option 2: Load from .txt file
# with open("LP-IV-datasets/CBOW/document1.txt", 'r', encoding='utf-8') as f:
#     data = f.read()

# ------------------------------------------------------------
# Clean and preprocess the text
# ------------------------------------------------------------
sentences = data.split('.')
clean_sent = []
for sentence in sentences:
    if sentence.strip() == "":
        continue
    sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence)
    sentence = re.sub(r'(?:^| )\w (?:$| )', ' ', sentence).strip()
    sentence = sentence.lower()
    clean_sent.append(sentence)

print("Total sentences:", len(clean_sent))
print("Sample sentence:", clean_sent[0] if clean_sent else "No valid text!")

# ------------------------------------------------------------
# b. Generate Training Data
# ------------------------------------------------------------
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_sent)
sequences = tokenizer.texts_to_sequences(clean_sent)

index_to_word = {}
word_to_index = {}

for i, sequence in enumerate(sequences):
    words = clean_sent[i].split()
    for j, value in enumerate(sequence):
        index_to_word[value] = words[j]
        word_to_index[words[j]] = value

vocab_size = len(tokenizer.word_index) + 1
emb_size = 10
context_size = 2

contexts, targets = [], []

for sequence in sequences:
    for i in range(context_size, len(sequence) - context_size):
        target = sequence[i]
        context = [
            sequence[i - 2], sequence[i - 1],
            sequence[i + 1], sequence[i + 2]
        ]
        contexts.append(context)
        targets.append(target)

X = np.array(contexts)
Y = np.array(targets)

print(f"Total training samples: {X.shape[0]} | Vocabulary size: {vocab_size}")

# ------------------------------------------------------------
# c. Train CBOW Model
# ------------------------------------------------------------
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=emb_size, input_length=2 * context_size),
    Lambda(lambda x: tf.reduce_mean(x, axis=1)),   # averages context vectors
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X, Y, epochs=60, verbose=1)
# --- Alternative (for faster testing): epochs=30 or batch_size=64 ---

# ------------------------------------------------------------
# d. Output and Visualization
# ------------------------------------------------------------

# Plot loss curve
plt.figure(figsize=(8,4))
sns.lineplot(x=range(len(history.history['loss'])),
             y=history.history['loss'], label='Training Loss')
plt.title("CBOW Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Print vocabulary and sample
print("\n📚 Vocabulary Sample:")
print(list(index_to_word.values())[:20])

# Visualize embeddings using PCA
embeddings = model.get_weights()[0]
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

# Test predictions for sample contexts
test_phrases = [
    "the speed of", 
    "transmission is an", 
    "influenza has a",
    "the serial interval"
]

print("\n🔍 Predicted words for sample contexts:")
for phrase in test_phrases:
    words = phrase.split()
    x_test = [word_to_index.get(i, 0) for i in words]
    x_test = np.array([x_test])
    pred = model.predict(x_test, verbose=0)
    pred_word = index_to_word.get(np.argmax(pred[0]), "unknown")
    print(f"Context: {words}  -->  Predicted center word: {pred_word}")

# Save model
model.save("cbow_model_generic.keras")

print("\n CBOW Model training completed successfully for this document.")
